## Goal: If we have 28 samples and have to choose to 

- 1. do lr rna-seq for 2 samples (only use GRCh38) or
- 2. do lr-rna-seq for 1 sample w/ personalized GRCh38 transcript discovery, what's best?

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import itertools
import os
import matplotlib.pyplot as plt
import yaml
from snakemake.io import expand
import pyranges as pr
from pyfaidx import Fasta
from mizani.formatters import percent_format
from scipy import stats


p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

from plotnine import *

In [2]:
def my_theme(base_size=11, w=4, h=3):
    """
    Custom plotnine theme with:
    - White background
    - Clean styling
    - Axes and ticks retained

    Parameters:
    - base_size: Base font size

    Returns:
    - plotnine.theme object
    """
    return (
        theme_minimal(base_size=base_size)
        + theme(
            # White background
            panel_background=element_rect(fill='white', color=None),
            plot_background=element_rect(fill='white', color=None),

            # Remove grid lines
            panel_grid_major=element_blank(),
            panel_grid_minor=element_blank(),
            panel_border=element_blank(),

            # Keep axis lines & ticks (don't blank them)
            axis_line=element_line(color='black'),
            axis_ticks=element_line(color='black'),

            plot_title=element_text(hjust=0.5, family='Helvetica'),
            axis_title_x=element_text(hjust=0.5, family='Helvetica'),
            axis_title_y=element_text(hjust=0.5, margin={'t':0, 'r':-2, 'b':0, 'l':0}, family='Helvetica'),
            
            # Styling text
            legend_title=element_blank(),
            axis_title=element_text(size=base_size + 1, family='Helvetica'),
            legend_text=element_text(size=base_size-2, family='Helvetica'),
            axis_text=element_text(size=base_size, color='black', family='Helvetica'),
            strip_text_x=element_text(size=base_size-1),
            strip_text_y=element_text(size=base_size-1),
            figure_size=(w, h),  # Controls plot dimensions (width x height in inches)
            plot_margin=0.05      # Shrinks surrounding white space
        )
    )

In [3]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [4]:
df = load_meta()
df = df.loc[~df['sample'].str.contains('_')]
df['lab_sample'] = df['lab_number_sample'].astype(str)+'_'+\
                      df['lab_sampleid'].astype(str)+'_'+\
                      df['cell_line_id'].astype(str)
df.columns
df = df[['cell_line_id', 'sample', 'hapmap_DNA_ID',
          'map_reads_assemblymap','lab_sample', 'population']].drop_duplicates()

temp_df = pd.read_csv('../snakemake/transcript_discovery_personal/cell_line_ids.txt', header=None, names=['cell_line_id'])

# make a 1000g cell line id col
df['cell_line_id_1000g'] = df.cell_line_id

inds = df.loc[~df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())].index
df.loc[inds, 'cell_line_id_1000g'] = df.loc[inds, 'hapmap_DNA_ID']
len(df.index)

# limit to just those in 1000g
df = df.loc[df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())]
assert len(df.index) == 30

# TODO bad sample that hasn't finished on espresso
# bad_samples = ['NA19328']
# df = df.loc[~df.cell_line_id_1000g.isin(bad_samples)]

hap = ['hap1', 'hap2']

meta_df = df.copy(deep=True)

In [5]:
file = proc_cfg(config['lr']['td_personal']['cerb']['ic_summary'],od)
df = pd.read_csv(file)
df.drop('Unnamed: 0', axis=1, inplace=True)
df['detected'] = True

df['n_ic'] = df.groupby('cell_line_id')['ic_id'].transform('nunique')         
df = df[['n_ic', 'cell_line_id']].drop_duplicates()

assert len(df.index) == 30
df = df.sort_values(by='n_ic', ascending=False)
cell_line_order = df.cell_line_id.tolist()
# cell_line_order

In [6]:
# def process_ic_summary(meta_df, 
#                        nov='all',
#                        genome='hg38',
#                        order=None):
#     """
#     Paramters:
#         nov (str): Novelty of reported ICs {'all', 'nov'}
#         genome (str): Genome for detection of ICs {'hg38', 'pers'}
#     Returns:
#         temp (pandas DataFrame): ??
#         big_df (pandas DataFrame): Some sort of summary dataframe
#         samples_2 (list of str): Cell line ids of 2 missing samples
#         samples_28 (list of str): Cell line ids of 28 included samples
#     """

#     file = proc_cfg(config['lr']['td_personal']['cerb']['ic_summary'],od)
#     df = pd.read_csv(file)
#     df.drop('Unnamed: 0', axis=1, inplace=True)
#     df['detected'] = True
    
#     # 1.5 only novel ics and count per cell line
#     if nov == 'nov':
#         print(len(df.index))
#         temp = df.loc[(df.structural_category!='full-splice_match')&\
#                       (df.structural_category!='incomplete-splice_match')]
#         print(len(temp.index))
#     elif nov == 'all':
#         temp = df.copy(deep=True)
    
#     # get only the detection from mapping
#     temp = temp[['ic_id', 'cell_line_id', 'map_genome', 'detected']]
#     temp = temp.drop_duplicates()
#     temp = temp.pivot(index=['ic_id', 'cell_line_id'], 
#                     columns=['map_genome'],
#                     values=['detected'])
    
#     # flatten
#     temp.columns = temp.columns.get_level_values(1)
#     temp.columns.name = None
    
#     # reset index to make it a flat DataFrame
#     temp = temp.reset_index()
    
#     # fill missing values with False
#     temp = temp.fillna(False)

    
#     if genome == 'pers':
#         # 1.75. get those that are detected in hap1/2 
#         temp = temp.loc[(temp.hap1+temp.hap2)>0]
#     elif genome == 'hg38': 
#         temp = temp.loc[(temp.hg38)==1]

#     temp['n_ic'] = temp.groupby('cell_line_id')['ic_id'].transform('nunique')         
#     temp2 = temp[['cell_line_id', 'n_ic']]

#     # order by # of detected ICs if no explicit order is given, otherwise
#     # order by the given order
#     if not order:
#         temp2 = temp2.sort_values(by='n_ic', ascending=False)
#     else:
#         temp2['cell_line_id'] = pd.Categorical(temp2['cell_line_id'], categories=order, ordered=True)
#         temp2 = temp2.sort_values('cell_line_id')
    
#     temp2.drop_duplicates(inplace=True)
#     # order = temp2.cell_line_id.tolist()
#     temp.drop(['hap1', 'hap2', 'hg38', 'n_ic'], axis=1, inplace=True)
    
#     temp['detected'] = True
#     temp = temp.pivot(index='ic_id', columns='cell_line_id')
#     temp.columns.name = None
#     temp.columns = temp.columns.get_level_values(1)
#     temp.fillna(False, inplace=True)
    
#     temp = temp[order]
#     prev_ics = []
#     big_df = pd.DataFrame()
#     for i, c in enumerate(temp.columns):
#         temp2 = temp.loc[temp[c]==True]
#         pop = meta_df.loc[meta_df.cell_line_id_1000g==c, 'population'].values[0]
#         ics = temp2.index.tolist()
#         n = len(list(set(ics)-set(prev_ics)))
#         prev_ics += ics
#         # print(len(prev_ics))
#         temp3 = pd.DataFrame(data=[[n,i,pop,c]], columns=['n_added_uniq', 'n_samples', 'population', 'cell_line_id_1000g'])
#         big_df = pd.concat([big_df, temp3], axis=0)
        
#     big_df['genome'] = genome
#     big_df['ic_nov'] = nov
#     big_df['n_cumulative_ics'] = big_df.n_added_uniq.cumsum()

#     samples_28 = order[:28]
#     samples_2 = order[-2:]
    
#     return temp, big_df, samples_28, samples_2

## All ICs, personal genomes

In [8]:
# temp, big_df, samples_28, samples_2 = process_ic_summary(meta_df, 
#                        nov='all',
#                        genome='pers', 
#                        order=cell_line_order)

In [10]:
file = proc_cfg(config['lr']['td_personal']['cerb']['ic_summary'],od)
df = pd.read_csv(file)
df['detected'] = True
df = df.loc[df.sqanti_genome=='hg38']

columns = ['sample_28_index',
           'start_n_reads',
           'n_ic_start', 
           'start_genome',
           'n_ic_new',
           'new_strat',
           'wgs_sample_2_index',
           'new_n_reads',
           'ic_nov']

df2 = pd.DataFrame()
           
def add_vals(df, vals, columns):
    df2 = pd.DataFrame(data=[vals], columns=columns)
    df = pd.concat([df, df2], axis=0)
    return df

# loop through each start genome
for start_genome in ['hg38', 'pers']:

    # loop through ic novelty
    for nov in ['all', 'nov']:
    
        # loop through each combination
        combinations = list(itertools.combinations(cell_line_order, 28))
        for i, comb in enumerate(combinations):
            
            # reads 28 samples (starting reads)
            n_reads = meta_df.loc[meta_df['cell_line_id_1000g'].isin(comb)]
            assert len(n_reads.index) == 28
            start_n_reads = n_reads['map_reads_assemblymap'].sum()
        
            # number of relevant ics found using LR-RNA-seq + GRCh38 
            # for the starting 28 samples
            if nov == 'all' and start_genome == 'hg38':
                temp = df.loc[(df.map_genome=='hg38')&\
                              (df.cell_line_id.isin(comb))]
            elif nov == 'nov' and start_genome == 'hg38':
                 temp = df.loc[(df.map_genome=='hg38')&\
                               (df.cell_line_id.isin(comb))&\
                               (df.structural_category!='full-splice_match')&\
                               (df.structural_category!='incomplete-splice_match')]
            elif nov == 'all' and start_genome == 'pers':
                temp = df.loc[(df.map_genome.isin(['hap2', 'hap1']))&\
                              (df.cell_line_id.isin(comb))]
            elif nov == 'nov' and start_genome == 'pers':
                 temp = df.loc[(df.map_genome.isin(['hap2', 'hap1']))&\
                               (df.cell_line_id.isin(comb))&\
                               (df.structural_category!='full-splice_match')&\
                               (df.structural_category!='incomplete-splice_match')]
            assert len(temp.cell_line_id.unique()) == 28
            start_ics = temp.ic_id.unique()
            n_start_ics = len(temp.ic_id.unique())
                                        
            # missing samples
            samples_2 = list(set(cell_line_order)-set(comb))
        
            # reads 2 samples (# added reads)
            n_reads = meta_df.loc[meta_df['cell_line_id_1000g'].isin(samples_2)]
            assert len(n_reads.index) == 2
            new_n_reads = n_reads['map_reads_assemblymap'].sum()
        
            # 1. adding 2 more samples by LR-RNA-seq only 
            # (ie only using GRCh38 as a reference)

            if nov == 'all':
                temp = df.loc[(df.map_genome=='hg38')&\
                              (df.cell_line_id.isin(samples_2))]
            elif nov == 'nov':
                temp = df.loc[(df.map_genome=='hg38')&\
                               (df.cell_line_id.isin(samples_2))&\
                               (df.structural_category!='full-splice_match')&\
                               (df.structural_category!='incomplete-splice_match')]
            assert len(temp.cell_line_id.unique()) == 2
            ics_2_rna = temp.ic_id.unique().tolist()
            
            vals = [i, start_n_reads,
                    n_start_ics, start_genome, 
               len(list(set(ics_2_rna)-set(start_ics))), # increase in number, set difference
               'rna_2_samples', np.nan, new_n_reads, nov]
            df2 = add_vals(df2, vals, columns)
        
            # 2. adding 1 more sample by LR-RNA-seq + WGS
            # (ie using personal GRCh38 only as reference)
            for j, c_id in enumerate(samples_2):
                # reads 1 sample1
                n_reads = meta_df.loc[meta_df['cell_line_id_1000g'].isin([c_id])]
                assert len(n_reads.index) == 1
                new_n_reads = n_reads['map_reads_assemblymap'].sum()

                if nov == 'all':
                    temp = df.loc[(df.cell_line_id.isin([c_id]))&\
                                   (df.map_genome.isin(['hap2', 'hap1']))]
                elif nov == 'nov':
                    temp = df.loc[(df.cell_line_id.isin([c_id]))&\
                                  (df.map_genome.isin(['hap2', 'hap1']))&\
                                  (df.structural_category!='full-splice_match')&\
                                  (df.structural_category!='incomplete-splice_match')]
                    
                ics_1_wgs = temp.ic_id.unique().tolist()
                vals = [i, start_n_reads,
                        n_start_ics, start_genome, 
                   len(list(set(ics_1_wgs)-set(start_ics))), # increase in number, set difference
                   'wgs_1_sample', j, new_n_reads, nov]
                df2 = add_vals(df2, vals, columns)
        

In [11]:
df2.to_csv('250714_28_sample_experiment.tsv', sep='\t')

In [15]:
len(df2[['sample_28_index', 'start_genome', 'ic_nov', 'wgs_sample_2_index']].drop_duplicates().index)

5220